In [1]:
def ReadFileIntoList(testFile = False):
    filename = 'input.txt'
    if testFile:
        filename = 'test.txt'
    with open(filename) as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]
    return lines

In [2]:
import pandas as pd
from collections import Counter

In [3]:
def GetRoutesDataFrame(testFile = False):
    input = ReadFileIntoList(testFile)
    fromList = [line.split('-')[0] for line in input]
    toList = [line.split('-')[1] for line in input]
    caves = fromList + toList
    caveList = list(set(caves))
    routes = []
    for i in range(len(caveList)):
        routes.append([False] * len(caveList))
    for src,dst in zip(fromList,toList):
        srcIndex = caveList.index(src)
        dstIndex = caveList.index(dst)
        routes[srcIndex][dstIndex] = True
        routes[dstIndex][srcIndex] = True
    df = pd.DataFrame(routes,index=caveList,columns=caveList)
    return df

In [4]:
def CreateCaveRoutesDict(df, currentLocation, route = [], canVisitSmallCave = False):
    cavesToGoTo = df.index[df.loc[currentLocation].values]
    routeDict = {}
    for cave in cavesToGoTo:
        if cave == 'end':
            routeDict['end'] = True
            continue
        if cave == 'start':
            continue
        if cave.islower() and cave in route:
            if not canVisitSmallCave:
                continue
            #potentialRoute = route.copy()
            #potentialRoute.append(cave)
            smallCaves = [c for c in route if c not in ['start','end'] and c.islower()]
            frequencies = list(Counter(smallCaves).values())
            frequencies.sort(reverse=True)
            if frequencies and frequencies[0] == 2:
                continue
        route.append(cave)
        routeDict[cave] = CreateCaveRoutesDict(df,cave, route, canVisitSmallCave)
        route.pop()
    return routeDict

In [5]:
def CreateCaveRouteList(df, currentLocation = 'start', canVisitSmallCaves = False):
    caveRoutesDict = {currentLocation:CreateCaveRoutesDict(df,currentLocation,[currentLocation],canVisitSmallCaves)}
    caveRouteDictList = pd.json_normalize(caveRoutesDict,sep='_').to_dict(orient='records')[0]
    caveRouteList = []
    for caveRoute in caveRouteDictList.keys():
        caveRouteList.append(caveRoute.split('_'))
    return caveRouteList

# Solution 1

In [6]:
df = GetRoutesDataFrame()
routeList = CreateCaveRouteList(df)
smallCaves = [cave for cave in df.index if cave not in ['start','end'] and cave.islower()]
smallCaveVisits = 0
for route in routeList:
    if [i for i in route if i in smallCaves]:
        smallCaveVisits += 1
print("Answer is", smallCaveVisits)

Answer is 3298


# Solution 2

In [7]:
df = GetRoutesDataFrame()
routeList = CreateCaveRouteList(df,canVisitSmallCaves=True)

print("Answer is", len(routeList))

Answer is 93572
